# 수업예제 함수를 사용해서 다시 해결하기

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
df_2021 = pd.read_csv("./data/kaggle/kaggle_survey_2021/kaggle_survey_2021_responses.csv")
df_2020 = pd.read_csv("./data/kaggle/kaggle_survey_2020/kaggle_survey_2020_responses.csv")
df_2019 = pd.read_csv("./data/kaggle/kaggle-survey-2019/multiple_choice_responses.csv")

df_2021.shape, df_2020.shape, df_2019.shape

((25974, 369), (20037, 355), (19718, 246))

In [5]:
print("2019:", df_2019['Q19'].unique().tolist())
print("2020:", df_2020['Q8'].unique().tolist())
print("2021:", df_2021['Q8'].unique().tolist())

2019: ['What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice', 'Python', nan, 'Java', 'R', 'SQL', 'C++', 'Other', 'C', 'MATLAB', 'TypeScript', 'Javascript', 'Bash']
2020: ['What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice', 'Python', 'R', 'C++', 'SQL', nan, 'Java', 'MATLAB', 'C', 'Other', 'Javascript', 'Julia', 'Swift', 'Bash']
2021: ['What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice', 'Python', 'SQL', 'R', 'MATLAB', 'C', 'Julia', nan, 'Other', 'C++', 'Javascript', 'Java', 'Bash', 'Swift']


In [7]:
programming_list = ["Python", "R", "SQL", "Java", "C", "Bash", "Javascript", "C++"]
programming_df = pd.Series(programming_list)

df_2019 = df_2019[df_2019['Q19'].isin(programming_df)]
df_2020 = df_2020[df_2020['Q8'].isin(programming_df)]
df_2021 = df_2021[df_2021['Q8'].isin(programming_df)]

print("2019:", df_2019['Q19'].unique().tolist())
print("2020:", df_2020['Q8'].unique().tolist())
print("2021:", df_2021['Q8'].unique().tolist())

2019: ['Python', 'Java', 'R', 'SQL', 'C++', 'C', 'Javascript', 'Bash']
2020: ['Python', 'R', 'C++', 'SQL', 'Java', 'C', 'Javascript', 'Bash']
2021: ['Python', 'SQL', 'R', 'C', 'C++', 'Javascript', 'Java', 'Bash']


In [8]:
q3_q5_q19_2019 = df_2019.loc[:, ['Q3', 'Q5', 'Q19']]
q3_q5_q19_2019 = q3_q5_q19_2019.rename(columns = {'Q19': 'Q8'}) # To match with other datasets

q3_q5_q8_2020 = df_2020.loc[:, ['Q3', 'Q5', 'Q8']]
q3_q5_q8_2021 = df_2021.loc[:, ['Q3', 'Q5', 'Q8']]

q3_q5_q19_2019['year'] = '2019'
q3_q5_q8_2020['year'] = '2020'
q3_q5_q8_2021['year'] = '2021'

final_df = pd.concat([q3_q5_q19_2019, q3_q5_q8_2020, q3_q5_q8_2021])
final_df.head()

,Q3,Q5,Q8,year
1,France,Software Engineer,Python,2019
4,Australia,Other,Python,2019
5,India,Other,Python,2019
6,France,Data Scientist,Java,2019
7,India,Data Scientist,Python,2019


In [13]:
import numpy as np
year_q5_q8 = final_df.groupby(['year', 'Q8']).size().reset_index().rename(columns = {0:"Count"})

year_q5_q8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    24 non-null     object
 1   Q8      24 non-null     object
 2   Count   24 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 704.0+ bytes


In [24]:
#연도별로 쪼개고, count, % 만드는 함수

def year_cut(df, year):     #year:문자형
    year_data = df[df['year'] == year].reset_index(drop=True)
    year_data['percent'] = year_data['Count'] / year_data['Count'].sum()
    year_data['%'] = np.round(year_data['percent']*100, 1)
    return year_data

df_2019 = year_cut(year_q5_q8, "2019")
df_2020 = year_cut(year_q5_q8, "2020")
df_2021 = year_cut(year_q5_q8, "2021")

df_2020

,year,Q8,Count,percent,%
0,2020,Bash,26,0.001507,0.2
1,2020,C,301,0.017443,1.7
2,2020,C++,325,0.018834,1.9
3,2020,Java,167,0.009678,1.0
4,2020,Javascript,88,0.005100,0.5
5,2020,Python,14241,0.825278,82.5
6,2020,R,1259,0.072960,7.3
7,2020,SQL,849,0.049200,4.9


In [40]:
# 그래프 한방에 그리기
import plotly.graph_objects as go

fig = go.Figure()

def create_bar_chart(df, year):
    fig.add_trace(go.Bar(x = df['Q8'], y = df['%'], name = str(year),
                        text=data['%'].astype(str) + "%", 
                         textposition='auto'))
    return fig

data_per_year = [(df_2019, 2019), (df_2020, 2020), (df_2021, 2021)]
for data, year in data_per_year:
    fig = create_bar_chart(data, year)

fig.show()